# Intel® SSD Data Center Tool Connector

This notebook demonstrates some of the quick analysis that can be done using the TOKIO connector for the Intel SSD Data Center Tool (ISDCT).  The format of the aggregated ISDCT outputs is specific to a tool developed at NERSC by David Paul and is therefore site-specific to NERSC, but the individual parsers for each ISDCT output file are generic.

In [ ]:
%matplotlib inline

In [ ]:
import os
import datetime
import numpy as np

import matplotlib
matplotlib.rcParams.update({'font.size': 18})
import matplotlib.pyplot as plt

import tokio.config
import tokio.connectors.nersc_isdct
import tokio.tools.common

In [ ]:
TARGET_DATE = datetime.datetime(year=2018, month=4, day=13)

GENERATE_PLOTS = True
PLOT_SUFFIX = "png" # or pdf, gif, jpeg...

In [ ]:
print "Generating report for %s" % TARGET_DATE.strftime('%c')
isdct_file = tokio.tools.common.enumerate_dated_files(start=TARGET_DATE,
                                                      end=TARGET_DATE,
                                                      template=tokio.config.ISDCT_FILES)
print "Using input file: %s" % isdct_file[0]

In [ ]:
isdct_data = tokio.connectors.nersc_isdct.NerscIsdct(isdct_file[0])
isdct_df = isdct_data.to_dataframe()

## Distribution of Lifetime Read/Write Loads

The following histograms demonstrate how many bytes have been written to and read from the SSD device _by applications_ over the entire service life of the SSD.

In [ ]:
for rw, column in ('read','data_units_read_bytes'), ('write', 'data_units_written_bytes'):
    fig, ax = matplotlib.pyplot.subplots()
    fig.set_size_inches(10, 6)
    fig.suptitle("%s Volume Distribution" % rw.title())

    ax.set_axisbelow(True)
    ax.grid(True)
    ax.set_xlabel("TiB %s" % rw.title())
    ax.set_ylabel("Number of SSDs")
    (isdct_df[column] / 2.0**40).hist(ax=ax, edgecolor='black')
    
    if GENERATE_PLOTS:
        output_file = 'histogram_%s_%s_%s.%s' % (rw, column, TARGET_DATE.strftime("%Y-%m-%d"), PLOT_SUFFIX)
        fig.savefig(output_file, bbox_inches='tight')
        print "Saved figure to", output_file

The read/write ratio from our applications should ideally match the read/write performance balance of the NVMe drives.  Writes are typically slower than reads on flash.

In [ ]:
fig, ax = matplotlib.pyplot.subplots(figsize=(10, 6))
ax.set_axisbelow(True)
ax.grid(True)
ax.set_xlabel("Read/Write Ratio")
ax.set_ylabel("Number of SSDs")

(isdct_df['data_units_read_bytes'] / isdct_df['data_units_written_bytes']).hist(ax=ax, edgecolor='black')

if GENERATE_PLOTS:
    output_file = 'histogram_readwrite_ratio.%s' % (PLOT_SUFFIX)
    fig.savefig(output_file, bbox_inches='tight')
    print "Saved figure to", output_file

## Write Amplification Distribution

Write amplification factor (WAF) is the ratio of bytes written to the device _by applications_ to the bytes written to the physical NAND chips, which includes both application-generated writes as well as writes caused by garbage collection.

A WAF of 1.0 is ideal; 2.0 is a normal level for the Intel SSDs we have in production.  High WAF is usually indicative of either

1. very new SSDs which have not seen much application-generated I/O; in these cases, the constant background load of the NVMe controller bubbles up to the surface

2. workloads which are very SSD-unfriendly.  These typically include writes that are not 4K aligned.  With DataWarp, the only non-4K aligned writes are those which are smaller than 4K.

In [ ]:
fig, ax = matplotlib.pyplot.subplots()
fig.set_size_inches(10, 6)
fig.suptitle("WAF Distribution")

ax.set_axisbelow(True)
ax.grid(True)
ax.set_xlabel("Write Amplification Factor")
ax.set_ylabel("Number of SSDs")
isdct_df['write_amplification_factor'].hist(ax=ax, edgecolor='black')

if GENERATE_PLOTS:
    output_file = 'histogram_waf_%s.%s' % (TARGET_DATE.strftime("%Y-%m-%d"), PLOT_SUFFIX)
    fig.savefig(output_file, bbox_inches='tight')
    print "Saved figure to", output_file

## Drive Writes per Day

Our Intel P3608 SSDs have a warranty of 5.0 drive writes per day (DWPD) when provisioned at 1.6 TB capacity for the five-year service life of the drive.

We have the option of reformatting the drives as 2.0 TB drives, which reduces the warranted endurance rating to 1.0 DWPD.

In [ ]:
fig, ax = matplotlib.pyplot.subplots()
fig.set_size_inches(10, 6)
fig.suptitle("DWPD Distribution")

ax.set_axisbelow(True)
ax.grid(True)
ax.set_xlabel("Drive Writes per Day")
ax.set_ylabel("Number of SSDs")
drive_writes = isdct_df['data_units_written_bytes'] / isdct_df['physical_size']
dwpd = drive_writes / isdct_df['power_on_hours'] * 24.0
dwpd.hist(ax=ax, edgecolor='black')

if GENERATE_PLOTS:
    output_file = 'histogram_dwpd_%s.%s' % (TARGET_DATE.strftime("%Y-%m-%d"), PLOT_SUFFIX)
    fig.savefig(output_file, bbox_inches='tight')
    print "Saved figure to", output_file

## Correlation Scatter Plots

Because many of the health metrics are ratios that get skewed when SSDs have seen very light use, it is sometimes helpful to correlate these health metrics with the number of hours the drives have been in service.

We expect the total volume of I/O to each drive to increase over time, and the WAF should decrease over time as each drive reaches steady state.

In [ ]:
scatter_plots = [
    ('power_on_hours', 'data_units_written_bytes'),
    ('power_on_hours', 'data_units_read_bytes'),
    ('power_on_hours', 'write_amplification_factor'),
]

In [ ]:
def scatter_and_fit_plot(df, x_key, y_key, fit=True):
    fig, ax = matplotlib.pyplot.subplots()
    fig.set_size_inches(10, 6)

    x = df[x_key].values
    y = df[y_key].values
    ax.plot(x, y, 'o', alpha=0.5)

    if fit:
        ### attempt a linear fit to generate a visual aid
        m, b = np.polyfit(x, y, 1)
        ax.plot(x, m*x+b, "-")

    ax.set_xlabel(x_key.replace('_', ' ').title())
    ax.set_ylabel(y_key.replace('_', ' ').title())
    plt.grid(True)
    if GENERATE_PLOTS:
        output_file = 'correlate_%s-%s_%s.%s' % (x_key, y_key, TARGET_DATE.strftime("%Y-%m-%d"), PLOT_SUFFIX)
        fig.savefig(output_file, bbox_inches='tight')
        print "Saved figure to", output_file

In [ ]:
for (x_key, y_key) in scatter_plots:
    scatter_and_fit_plot(isdct_df, x_key, y_key)

## Identify faulty node power sources

The "PLI Lock Loss" counter was originally thought to be an indicator of unhealthy drives.  It turns out that this metric is really a PLL (phase-locked loop) lock loss count, which increments when the PCIe timing signal falls irreparably out of sync with the internal clock on the SSD.  This is __not__ an indicator of bad drive health as originally thought; it is an indicator of unclean power to the host node.

In [ ]:
pli_lock_losses = isdct_df[isdct_df['smart_pli_lock_loss_count_raw'] > 0]
pli_lock_losses[['node_name', 'smart_pli_lock_loss_count_raw', 'power_on_hours']]\
    .sort_values('smart_pli_lock_loss_count_raw', ascending=False)

In [ ]:
x_key = 'power_on_hours'
y_key = 'smart_pli_lock_loss_count_raw'
fig, ax = matplotlib.pyplot.subplots()
fig.set_size_inches(10, 6)

ax.plot(isdct_df[x_key].values,
        isdct_df[y_key].values,
        marker='o',
        linestyle='none',
        alpha=0.5,
        label="All SSDs")
ax.plot(pli_lock_losses[x_key],
        pli_lock_losses[y_key],
        marker='o',
        linestyle='none',
        alpha=0.5,
        color='red',
        markersize=10,
        markerfacecolor='none',
        label="Nonzero PLI Lock Loss")

ax.legend()
ax.set_xlabel(x_key.replace('_', ' ').title())
ax.set_ylabel(y_key.replace('_', ' ').title())
plt.grid(True)

if GENERATE_PLOTS:
    output_file = 'lockloss_vs_poweron_%s.%s' % (TARGET_DATE.strftime("%Y-%m-%d"), PLOT_SUFFIX)
    fig.savefig(output_file, bbox_inches='tight')
    print "Saved figure to", output_file